In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.nn import Linear, Sequential, ReLU
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam, AdamW
import torchinfo
from torchinfo import summary

import os
import tqdm
from tqdm.auto import tqdm, trange

import torchvision
from torchvision.datasets import ImageFolder
from torchvision.transforms import v2
import torchmetrics
from torchmetrics import Accuracy

import PIL
from PIL import Image
import shutil

import torch.nn as nn
import torch.optim as optim

c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
train_ = pd.read_csv("C:/Users/Administrator/Documents/PMLDL2024/Assignment 1/PMLD-Assignment-1/code/datasets/Training_set.csv")

In [3]:
train_.head()

,filename,label
0,Image_1.jpg,SOUTHERN DOGFACE
1,Image_2.jpg,ADONIS
2,Image_3.jpg,BROWN SIPROETA
3,Image_4.jpg,MONARCH
4,Image_5.jpg,GREEN CELLED CATTLEHEART


In [4]:
def to_lower(col):
    return col.lower()

train_["label"] = train_["label"].apply(to_lower)
classes_ = [i for i in train_["label"].unique()]
train_.shape

(6499, 2)

In [5]:
for i in tqdm(classes_):
    os.makedirs(f"C:/Users/Administrator/Documents/PMLDL2024/Assignment 1/PMLD-Assignment-1/code/models/train/{i}")
    os.makedirs(f"C:/Users/Administrator/Documents/PMLDL2024/Assignment 1/PMLD-Assignment-1/code/models/val/{i}")

  0%|          | 0/75 [00:00<?, ?it/s]

In [6]:
train_split, val_split = train_test_split(train_, test_size = 0.15, stratify = train_["label"])
train_split.shape, val_split.shape

((5524, 2), (975, 2))

In [7]:
def copy_files(df_:pd.DataFrame, img_path_:str, dest_str):
    for i in tqdm(classes_):
        dest_ = dest_str+"/"+i
        df_class = df_[df_["label"] == i]
        os.chdir(img_path_)
        for j in df_class["filename"].values:
            shutil.copy(j, dest_)

copy_files(val_split, "C:/Users/Administrator/Documents/PMLDL2024/Assignment 1/PMLD-Assignment-1/code/datasets/train", "C:/Users/Administrator/Documents/PMLDL2024/Assignment 1/PMLD-Assignment-1/code/models/val")
copy_files(train_split, "C:/Users/Administrator/Documents/PMLDL2024/Assignment 1/PMLD-Assignment-1/code/datasets/train", "C:/Users/Administrator/Documents/PMLDL2024/Assignment 1/PMLD-Assignment-1/code/models/train")

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

In [8]:
for i in os.listdir("C:/Users/Administrator/Documents/PMLDL2024/Assignment 1/PMLD-Assignment-1/code/models/val"):
    print(f"{i} => {len(os.listdir(f'C:/Users/Administrator/Documents/PMLDL2024/Assignment 1/PMLD-Assignment-1/code/models/val/{i}'))}")

adonis => 13
african giant swallowtail => 11
american snoot => 11
an 88 => 13
appollo => 14
atala => 15
banded orange heliconian => 15
banded peacock => 12
beckers white => 12
black hairstreak => 13
blue morpho => 11
blue spotted crow => 13
brown siproeta => 15
cabbage white => 14
cairns birdwing => 12
checquered skipper => 14
chestnut => 13
cleopatra => 14
clodius parnassian => 13
clouded sulphur => 14
common banded awl => 13
common wood-nymph => 14
copper tail => 14
crecent => 15
crimson patch => 11
danaid eggfly => 14
eastern coma => 14
eastern dapple white => 14
eastern pine elfin => 14
elbowed pierrot => 12
gold banded => 11
great eggfly => 12
great jay => 14
green celled cattleheart => 13
grey hairstreak => 13
indra swallow => 12
iphiclus sister => 14
julia => 12
large marble => 12
malachite => 11
mangrove skipper => 13
mestra => 13
metalmark => 11
milberts tortoiseshell => 14
monarch => 14
mourning cloak => 20
orange oakleaf => 13
orange tip => 14
orchard swallow => 11
painted l

In [9]:
transform_train = v2.Compose([
    v2.Resize((128,128)),
    v2.RandomAffine(degrees = (-50,+50)),
    v2.RandomRotation(15),
    v2.RandomHorizontalFlip(0.3),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale = True),
    v2.Normalize(mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225]),
])
transform_val = v2.Compose([
    v2.Resize((128,128)),
    v2.ToImage("RGB"),
    v2.ToDtype(torch.float32, scale = True),
    v2.Normalize(mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225]),
])

In [10]:
train_ = ImageFolder("C:/Users/Administrator/Documents/PMLDL2024/Assignment 1/PMLD-Assignment-1/code/models/train",
                     transform = transform_train)
val_ = ImageFolder("C:/Users/Administrator/Documents/PMLDL2024/Assignment 1/PMLD-Assignment-1/code/models/val",
                     transform = transform_val)

In [11]:
BATCH_SIZE = 32
train_dataloader = DataLoader(dataset = train_,
                              batch_size = BATCH_SIZE,
                              shuffle = True,
                              num_workers = os.cpu_count(),
                              persistent_workers = True)

val_dataloader = DataLoader(dataset = val_,
                            batch_size = BATCH_SIZE,
                            shuffle = False,
                            num_workers = os.cpu_count(),
                            persistent_workers = True)

In [12]:
device = "cuda" 

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

class ButterCLF(nn.Module):
    def __init__(self, num_classes):
        super(ButterCLF, self).__init__()
        
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.block4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.fc = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


In [14]:
num_classes = 75
model = ButterCLF(num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, model_save_path="C:/Users/Administrator/Documents/PMLDL2024/Assignment 1/PMLD-Assignment-1/code/models/best.pt"):
    best_val_acc = 0.0

    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        val_acc = validate_model(model, val_loader)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), model_save_path)  

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}, Val Accuracy: {val_acc}')


def validate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

train_model(model, train_dataloader, val_dataloader, criterion, optimizer, num_epochs=100)

Epoch [1/100], Loss: 3.763724435960626, Val Accuracy: 12.512820512820513
Epoch [2/100], Loss: 3.0892029098003584, Val Accuracy: 21.743589743589745
Epoch [3/100], Loss: 2.6807738921545834, Val Accuracy: 27.487179487179485
Epoch [4/100], Loss: 2.340122698359407, Val Accuracy: 34.15384615384615
Epoch [5/100], Loss: 2.124245918555067, Val Accuracy: 36.205128205128204
Epoch [6/100], Loss: 1.9206433365110718, Val Accuracy: 46.666666666666664
Epoch [7/100], Loss: 1.7159948989835088, Val Accuracy: 49.02564102564103
Epoch [8/100], Loss: 1.5376157863980773, Val Accuracy: 48.41025641025641
Epoch [9/100], Loss: 1.3980506172758995, Val Accuracy: 53.64102564102564
Epoch [10/100], Loss: 1.3032093375404445, Val Accuracy: 55.58974358974359
Epoch [11/100], Loss: 1.1665624921032458, Val Accuracy: 56.717948717948715
Epoch [12/100], Loss: 1.061567420904347, Val Accuracy: 58.76923076923077
Epoch [13/100], Loss: 0.9933938566659917, Val Accuracy: 55.282051282051285
Epoch [14/100], Loss: 0.9020066283686312, Va